# Week 11

In this week, we learn the difference between SQL 
and PySpark and also how to use SQL in PySpark.   
At the end we will understand how to write purely PySpark-style functions
for performing queries, purely SQL-style operators, and mixed both them.

In [1]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException

In [2]:
spark = (SparkSession
  .builder
  .master("local[*]")   # optional
  .appName("Bilingual PySpark")
  .getOrCreate())

In [3]:
spark

## A demonstration between PySpark functions and ANSI SQL

For ANSI SQL, we use PostgreSQL as its implementation. 
If you want to learn or review SQL in quick way, please check
my repo [AK-201-426-data-science/ch-24-(pgAdmin)-database-and-sql](
https://github.com/LugoBlogger/AK-201-426-data-science/blob/master/ch-24-(pgAdmin)-database-and-sql.md). There is also a video tutorial in my Youtube channel
[AK-201-426 - Data Science - Week 15 - SQL](https://youtu.be/YlaQyeHgBUg)

First we use the following elements table to show the difference in sytanx
between SQL and PySpark


<img src="./img-resources/07-elements-table.png" width=1000>

**SQL in PostgreSQL**

```sql
SELECT "Period", COUNT(*) 
  FROM periodic_table
  WHERE "Phase" = 'liq'
  GROUP BY "Period";
```

<img src="./img-resources/07-elements-table-sql-results.png" width=400>

**PySpark functions**

In [4]:
elements = spark.read.csv(
  "./data/elements/Periodic_Table_Of_Elements.csv",
  header=True, 
  inferSchema=True)

elements.show(5, truncate=10)
elements.printSchema()

+------------+---------+------+----------+----------------+---------------+-----------------+------+-----+-----+-----------+-------+-----+--------+---------+----------+------------+-----------------+---------------+-------+------------+------------+----------------+----------+----+------------+--------------+---------------+
|AtomicNumber|  Element|Symbol|AtomicMass|NumberofNeutrons|NumberofProtons|NumberofElectrons|Period|Group|Phase|Radioactive|Natural|Metal|Nonmetal|Metalloid|      Type|AtomicRadius|Electronegativity|FirstIonization|Density|MeltingPoint|BoilingPoint|NumberOfIsotopes|Discoverer|Year|SpecificHeat|NumberofShells|NumberofValence|
+------------+---------+------+----------+----------------+---------------+-----------------+------+-----+-----+-----------+-------+-----+--------+---------+----------+------------+-----------------+---------------+-------+------------+------------+----------------+----------+----+------------+--------------+---------------+
|           1| Hydr

In [5]:
(elements
  .where(F.col("phase") == "liq")
  .select(F.col("Element"))
  .show())

+-------+
|Element|
+-------+
|Bromine|
|Mercury|
+-------+



In [6]:
(elements
  .where(F.col("phase") == "liq")
  .groupby("period")
  .count()
  .show())

+------+-----+
|period|count|
+------+-----+
|     6|    1|
|     4|    1|
+------+-----+



**Embedded SQL in PySpark**

PySpark doesn't make the link between the Python variable `elements` automatically

In [7]:
# to drop `elements` from PySpark view
spark.catalog.dropTempView("elements") 

try:
  spark.sql("""
    SELECT period, COUNT(*) 
      FROM elements 
      WHERE phase='liq' 
      GROUP BY period
  """).show(5)
except AnalysisException as e:
  print(e)

[TABLE_OR_VIEW_NOT_FOUND] The table or view `elements` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 3 pos 11;
'Aggregate ['period], ['period, unresolvedalias(count(1), None)]
+- 'Filter ('phase = liq)
   +- 'UnresolvedRelation [elements], [], false



Create a link of to `elements` Python variable to internal "elements"
that can be accessed with embedded SQL commands in PySpark

In [8]:
elements.createOrReplaceTempView("elements")

spark.sql("""
  SELECT period, COUNT(*) 
    FROM elements 
    WHERE phase='liq' 
    GROUP BY period
""").show(5)

+------+--------+
|period|count(1)|
+------+--------+
|     6|       1|
|     4|       1|
+------+--------+



## Processing big data with PySpark SQL

We use the datasets from [Backblaze](https://www.backblaze.com/cloud-storage/resources/hard-drive-test-data#downloadingTheRawTestData)   

The fullset reading and union takes 41 secs. 
For a single reading `data_Q3_2019` only takes 8 secs.

In [9]:
use_fullset = True # default is False

# DATA_DIRECTORY = "./data/backblaze/"   # change into this for the students
DATA_DIRECTORY = "../rioux-2022/data/backblaze/"

# 8 secs
print("Read data_Q3_2019")
q3 = spark.read.csv(
  DATA_DIRECTORY + "data_Q3_2019", header=True, inferSchema=True)


if use_fullset:
  print("Read data_Q1_2019")
  q1 = spark.read.csv(
    DATA_DIRECTORY + "data_Q1_2019", header=True, inferSchema=True)
  
  print("Read data_Q2_2019")
  q2 = spark.read.csv(
    DATA_DIRECTORY + "data_Q2_2019", header=True, inferSchema=True)
  
  print("Read data_Q4_2019")
  q4 = spark.read.csv(
    DATA_DIRECTORY + "data_Q4_2019", header=True, inferSchema=True)
  
  # Q4 has two more fields than the rest
  q4_fields_extra = set(q4.columns) - set(q1.columns)

  for i in q4_fields_extra:
    q1 = q1.withColumn(i, F.lit(None).cast(T.StringType()))
    q2 = q2.withColumn(i, F.lit(None).cast(T.StringType()))
    q3 = q3.withColumn(i, F.lit(None).cast(T.StringType()))

  # if you are using the full set of data, use this version
  backblaze_2019 = (
    q1.select(q4.columns)
      .union(q2.select(q4.columns))
      .union(q3.select(q4.columns))
      .union(q4.select(q4.columns))
  )
else:
  # if you are only using the minimal set of data, use this version
  backblaze_2019 = q3


# Setting the layout for each column according to the schema
backblaze_2019 = backblaze_2019.select(
  [F.col(x).cast(T.LongType()) if x.startswith("smart") else F.col(x)
    for x in backblaze_2019.columns]
)

# Connect Python variable backblaze_2019 to PySpark SQL table `backblaze_stats_2019`
backblaze_2019.createOrReplaceTempView("backblaze_stats_2019")

Read data_Q3_2019
Read data_Q1_2019
Read data_Q2_2019
Read data_Q4_2019


The meaning of the columns of S.M.A.R.T. stats can be checked in 
[ATA S.M.A.R.T. attributes](https://en.wikipedia.org/wiki/Self-Monitoring,_Analysis_and_Reporting_Technology)


In [13]:
backblaze_2019.show(10)

+----------+--------------+--------------------+--------------+-------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+---

There are 10,338,153 rows in `data_Q3_2019`

In [ ]:
total_row = backblaze_2019.count()
f"{total_row:,}"

There are 116,270 distinct `serial_number` in `data_Q3_2019`

In [ ]:
unique_serial_number = backblaze_2019.select(F.col("serial_number")).distinct().count()
f"{unique_serial_number:,}"

There are 43 distinct `model` in `data_Q3_2019`

In [ ]:
df_unique_model = backblaze_2019.select(F.col("model")).distinct()

In [ ]:
df_unique_model.show(50, truncate=False)
df_unique_model.count()

**Self-exercise**

Create a new column `mfg` that represents the manufacturing company of the hard-drive:  
- `ST` or `Se`: "Seagate"
- `HG`: "HGST"
- `Hi`: "HGST"
- `TO`: "Toshiba"
- `WD`: "WDC"
- `DE`: "Dell"

Manual solution

In [ ]:
(df_unique_model
  .select(
    F.col("model"), F.substring(F.col("model"), 1, 2).alias("first2"))
  .select(
    F.col("model"), 
    (F.when(F.col("first2") == "ST", "Seagate")
      .otherwise(F.col("first2")).alias("first2"))
  )
  .select(
    F.col("model"), 
    (F.when(F.col("first2") == "Se", "Seagate")
      .otherwise(F.col("first2")).alias("first2"))
  )
  .select(
    F.col("model"), 
    (F.when(F.col("first2") == "TO", "Toshiba")
      .otherwise(F.col("first2")).alias("first2"))
  )
  .select(
    F.col("model"), 
    (F.when(F.col("first2") == "WD", "WDC")
      .otherwise(F.col("first2")).alias("first2"))
  )
).show(50, truncate=False)

With the help of ChatGPT

In [ ]:
df_unique_model.withColumn(
  "mfg", 
  F.when(F.col("model").startswith("ST"), "Seagate")
   .when(F.col("model").startswith("Se"), "Seagate")
   .when(F.col("model").startswith("TO"), "Toshiba")
   .when(F.col("model").startswith("WD"), "WDC")
   .when(F.col("model").startswith("HG"), "HGST")
   .when(F.col("model").startswith("Hi"), "HGST")
   .when(F.col("model").startswith("DE"), "DELL")
).show(50, truncate=False)

## Playing around with Spark SQL and PySpark on `backblaze_stats_2019`

### Show only the failure serial numbers

**Embedded SQL in PySpark**

In [10]:
# data_Q3_2019: 622 rows for the failure state (.count())
spark.sql('''
  SELECT serial_number 
    FROM backblaze_stats_2019 
    WHERE failure = 1
''').show(5)

+--------------+
| serial_number|
+--------------+
|     57GGPD9NT|
|      ZJV03Y00|
|      ZDEB33GK|
|      Z302T6CW|
|PL1331LAHBYKEH|
+--------------+
only showing top 5 rows



**PySpark functions**

In [12]:
(backblaze_2019
  .select(F.col("serial_number"))
  .where(F.col("failure") == 1)
  .show(5))

print((backblaze_2019
  .select(F.col("serial_number"))
  .where(F.col("failure") == 1)
).count())

print((backblaze_2019
  .select(F.col("serial_number"))
  .where(F.col("failure") == 1)
).distinct().count())


+--------------+
| serial_number|
+--------------+
|     57GGPD9NT|
|      ZJV03Y00|
|      ZDEB33GK|
|      Z302T6CW|
|PL1331LAHBYKEH|
+--------------+
only showing top 5 rows

2226
2226


### Grouping and ordering

Let us see the `capacity_bytes` column

In [14]:
backblaze_2019.select(
  F.col("capacity_bytes")
).show(5)


# data_Q3_2019: distinct row of `capacity_bytes`: 14
backblaze_2019.select(
  F.col("capacity_bytes")
).distinct().count()

+--------------+
|capacity_bytes|
+--------------+
| 4000787030016|
|12000138625024|
|12000138625024|
|12000138625024|
| 4000787030016|
+--------------+
only showing top 5 rows



17

Let us see the `model` name column

In [15]:
backblaze_2019.select(
  F.col("model")
).show(5)

backblaze_2019.select(
  F.col("model")
).distinct().count()

+--------------------+
|               model|
+--------------------+
|         ST4000DM000|
|       ST12000NM0007|
|       ST12000NM0007|
|       ST12000NM0007|
|HGST HMS5C4040ALE640|
+--------------------+
only showing top 5 rows



55

**Embedded SQL in PySpark**

In [17]:
# Approximately 9 secs for all quarters
spark.sql("""
  SELECT 
    model,
    MIN(capacity_bytes / POW(1024, 3)) min_GB,
    MAX(capacity_bytes / POW(1024, 3)) max_GB
  FROM backblaze_stats_2019
  GROUP BY 1 
  ORDER BY 3 DESC
  """).show(5, truncate=False)

+--------------------+----------------------+-------+
|model               |min_GB                |max_GB |
+--------------------+----------------------+-------+
|ST16000NM001G       |14902.0               |14902.0|
|TOSHIBA MG07ACA14TA |-9.313225746154785E-10|13039.0|
|ST12000NM0007       |-9.313225746154785E-10|11176.0|
|HGST HUH721212ALE600|11176.0               |11176.0|
|HGST HUH721212ALN604|-9.313225746154785E-10|11176.0|
+--------------------+----------------------+-------+
only showing top 5 rows



**PySpark functions**

In [18]:
(backblaze_2019
  .groupby(F.col("model"))
  .agg(
    F.min(F.col("capacity_bytes") / F.pow(F.lit(1024), 3)).alias("min_GB"),
    F.max(F.col("capacity_bytes") / F.pow(F.lit(1024), 3)).alias("max_GB"))
  .where(F.col("min_GB") != F.col("max_GB"))
  .orderBy(F.col("max_GB"), ascending=False)
  .show(5))

+--------------------+--------------------+-------+
|               model|              min_GB| max_GB|
+--------------------+--------------------+-------+
| TOSHIBA MG07ACA14TA|-9.31322574615478...|13039.0|
|       ST12000NM0007|-9.31322574615478...|11176.0|
|HGST HUH721212ALN604|-9.31322574615478...|11176.0|
|HGST HUH721010ALE600|-9.31322574615478...| 9314.0|
|       ST10000NM0086|-9.31322574615478...| 9314.0|
+--------------------+--------------------+-------+
only showing top 5 rows



In [ ]:
spark = SparkSession.builder.getOrCreate()

DATA_DIRECTORY = "./data/backblaze/"
DATA_FILES = [
  "data_Q1_2019",
  "data_Q2_2019",
  "data_Q3_2019",
  "data_Q4_2019"]

data = [spark.read.csv(DATA_DIRECTORY + file, header=True, inferSchema=True)
        for file in DATA_FILES]
common_columns = list(
  reduce(lambda x, y: x.intersection(y), [set(df.columns) for df in data])
)

assert set(["model", "capacity_bytes", "date", "failure"]).issubset(
  set(common_columns)
)

full_data = reduce(
  lambda x, y: x.select(common_columns).union(y.select(common_columns)), data)

In [ ]:
full_data_selected = full_data.selectExpr(
  "model", "capacity_bytes / pow(1024, 3) capacity_GB", "date", "failure")

drive_days = (full_data_selected
  .groupby("model", "capacity_GB")
  .agg(F.count("*").alias("drive_days")))

failures = (full_data_selected
  .where("failure = 1")
  .groupby("model", "capacity_GB")
  .agg(F.count("*").alias("failures")))

summarized_data = (drive_days
  .join(failures, on=["model", "capacity_GB"], how="left")
  .fillna(0.0, ["failures"])
  .selectExpr("model", "capacity_GB", "failures / drive_days failure_rate")
  .cache())

In [ ]:
def most_reliable_drive_for_capacity(data, capacity_GB=2048, precision=0.25, top_n=3):
  """Return the top 3 drives for a given approximate capacity.
  Given a capacity in GB and a precision as a decimal number, we keep the N
  drives where:
  - the capacity is between (capacity * 1/(1+precision)), capacity * (1+precision) 
  - the failure rate is the lowest
  """ 
  capacity_min = capacity_GB / (1 + precision)
  capacity_max = capacity_GB * (1 + precision)

  # in .where, we use string interpolation for extending the capability of
  # SQL-style operations
  answer = (data
    .where(f"capacity_GB between {capacity_min} and {capacity_max}")
    .orderBy("failure_rate", "capacity_GB", ascending=[True, False])
    .limit(top_n)
  )

  return answer

In [ ]:
most_reliable_drive_for_capacity(summarized_data, capacity_GB=11176.0).show()